In [1]:
!pip install tensorflow==2.8.0

     |████████████████████████████████| 462 kB 30.0 MB/s 


In [2]:
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
# TODO: Change Models You want https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet
from tensorflow.keras.applications.xception import Xception
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
import numpy as np

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.8.0


## **CIFAR 10 Datast 데이터셋 다운로드하고 준비하기**




In [3]:
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  # TODO: Change Models You want to change
  output_ims = tf.keras.applications.xception.preprocess_input(input_images)
  return output_ims

In [4]:
(training_images, training_labels) , (validation_images, validation_labels) = tf.keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 13s 0us/step


In [5]:
train_X = preprocess_image_input(training_images)
valid_X = preprocess_image_input(validation_images)

In [6]:
train_X.shape

(50000, 32, 32, 3)

In [7]:
'''
Feature Extraction is performed by Xception pretrained on imagenet weights. 
Input size is 224 x 224.
'''
def feature_extractor(inputs):

  #Change Models Here too!!!
  
  feature_extractor = tf.keras.applications.xception.Xception(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
  return feature_extractor


'''
Defines final dense layers and subsequent softmax layer for classification.
'''
def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)
    return x

'''
Since input image size is (32 x 32), first upsample the image by factor of (7x7) to transform it to (224 x 224)
Connect the feature extraction and "classifier" layers to build the model.
'''
def final_model(inputs):

    resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)
    xception_feature_extractor = feature_extractor(resize)
    classification_output = classifier(xception_feature_extractor)

    return classification_output

'''
Define the model and compile it. 
Use Stochastic Gradient Descent as the optimizer.
Use Sparse Categorical CrossEntropy as the loss function.
'''
def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(32,32,3))
  
  classification_output = final_model(inputs) 
  model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
  model.compile(optimizer='Adam', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model


model = define_compile_model()

model.summary()

83697664/83683744 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                              

In [8]:
EPOCHS = 3
history = model.fit(train_X, training_labels, epochs=EPOCHS, validation_data = (valid_X, validation_labels), batch_size=64)

Epoch 1/3
782/782 [==============================] - 1027s 1s/step - loss: 0.4703 - accuracy: 0.8538 - val_loss: 0.5399 - val_accuracy: 0.8226
Epoch 2/3
782/782 [==============================] - 1032s 1s/step - loss: 0.2562 - accuracy: 0.9201 - val_loss: 0.3563 - val_accuracy: 0.8910
Epoch 3/3
782/782 [==============================] - 1031s 1s/step - loss: 0.1873 - accuracy: 0.9394 - val_loss: 0.3416 - val_accuracy: 0.8923


In [9]:
loss, accuracy = model.evaluate(valid_X, validation_labels, batch_size=64)

157/157 [==============================] - 43s 275ms/step - loss: 0.3416 - accuracy: 0.8923
